In [4]:
import numpy as np
import pprint
from simpsons import simpsons_integration
from simple import simple_integration

pp = pprint.PrettyPrinter(indent=4)


# Tests

We're going to test a list of functions with well-known integration values within the range [a,b]. 
Test known integrals, with variable dx belonging to 
Moon period is 2358720s. 1s of this is 0.000042% of the orbit. We test with dx

[0.000042, 0.00042, 0.0042, 0.042] (%). 



1. Straight lines

    $ y = x $

2. Quadratics

    $ y = x^2 $

3. Exponentials

    $ y = e^x $

4. Ellipse

    $ y = sin(x) $


In [14]:
class TestIntegrationMethods:
    def __init__(self):
        self.a = 0
        self.b = 10
        self.areas = {
            "straight_line": {
                "actual": 50,
                "simple": None,
                "simpsons": None,
                "function": self.straight_lines_func
            },
            "quadratics": {
                "actual": 1000/3,
                "simple": None,
                "simpsons": None,
                "function": self.quadratic_func
            },
            "exponentials": {
                "actual": np.exp(10)-np.exp(0),
                "simple": None,
                "simpsons": None,
                "function": self.exponentials_func
            },
            "sine": {
                "actual": np.cos(0)-np.cos(10),
                "simple": None,
                "simpsons": None,
                "function": self.sine_func
            }
        }
        self.dx = np.array(
            [4.24e-7, 4.24e-6, 4.24e-5, 4.24e-4])*(self.b-self.a)  # dx
        self.n = np.array([np.int64((self.b-self.a)//dx_i)
                          for dx_i in self.dx])  # no. of partitions
        self.xs = [np.linspace(self.a, self.b, n)
                   for n in self.n]  # x values of equal width

        self.main()

    def straight_lines_func(self, x):
        return x

    def quadratic_func(self, x, a=1, b=0, c=0):
        return a*x**2 + b*x + c

    def exponentials_func(self, x):
        return np.exp(x)

    def sine_func(self, x):
        return np.sin(x)

    def integral_test(self, integral_method, f):
        total_area = 0
        for i in range(0, len(self.xs[0])-1):
            x0 = self.xs[0][i]
            # TODO change back index to j for looping through intervals
            x2 = self.xs[0][i+1]
            total_area += integral_method(f, x0, x2)
        return total_area

    def get_areas(self, line):
        simpsons_area = self.integral_test(simpsons_integration, line)
        simple_area = self.integral_test(simple_integration, line)
        return {"simple": simple_area, "simpsons": simpsons_area}

    def main(self):
        for name, equation_type in self.areas.items():
            print(f'Starting: {name}')
            solutions = self.get_areas(equation_type['function'])

            equation_type['simpsons'] = solutions['simpsons']
            equation_type['simple'] = solutions['simple']
            print(f'Completed: {name}')


In [15]:
results = TestIntegrationMethods()

pp.pprint(results.areas)


Starting: straight_line
Completed: straight_line
Starting: quadratics
Completed: quadratics
Starting: exponentials
Completed: exponentials
Starting: sine
Completed: sine
{   'exponentials': {   'actual': 22025.465794806718,
                        'function': <bound method TestIntegrationMethods.exponentials_func of <__main__.TestIntegrationMethods object at 0x7f22e49aa850>>,
                        'simple': 22025.419100821066,
                        'simpsons': 22025.465794806987},
    'quadratics': {   'actual': 333.3333333333333,
                      'function': <bound method TestIntegrationMethods.quadratic_func of <__main__.TestIntegrationMethods object at 0x7f22e49aa850>>,
                      'simple': 333.3331213332305,
                      'simpsons': 333.33333333331564},
    'sine': {   'actual': 1.8390715290764525,
                'function': <bound method TestIntegrationMethods.sine_func of <__main__.TestIntegrationMethods object at 0x7f22e49aa850>>,
                's

In [18]:
def print_results():
    for name, data in results.areas.items():
        true_value = data['actual']
        simple_diff = abs(true_value - data['simple'])/true_value * 100
        simpsons_diff =  abs(true_value - data['simpsons'])/true_value * 100
        print(f'Equation: {name}')
        print(f'Simple % error: {simple_diff}%')
        print(f'Simpsons % error: {simpsons_diff}%')
        
        print(f'Simple Error / Simpsons Error : {simple_diff/simpsons_diff}')
print_results()
    

Equation: straight_line
Simple % error: 4.240002824928979e-05%
Simpsons % error: 7.105427357601002e-14%
Simple Error / Simpsons Error : 596727348.2
Equation: quadratics
Simple % error: 6.360003084182608e-05%
Simpsons % error: 5.3034909797133884e-12%
Simple Error / Simpsons Error : 11992106.913183277
Equation: exponentials
Simple % error: 0.00021199999167959485%
Simpsons % error: 1.222268960088293e-12%
Simple Error / Simpsons Error : 173447905.9864865
Equation: sine
Simple % error: 6.271223690007873e-05%
Simpsons % error: 3.501382866853255e-13%
Simple Error / Simpsons Error : 179107053.6551724


It's clear that when the true value is accurate, the Simpsons outperforms the simple integration techique massively. What happens when the true integration value is a little out?

In [23]:
def adjust_percent_up(percent):
    multiplier = 1 + percent/100
    for _,item in results.areas.items():
        item['actual'] *=multiplier

def remove_adjustment(percent):
    multiplier = 1/(1 + percent/100)
    for _, item in results.areas.items():
        item['actual'] *= multiplier


adjust_percent_up(10)
print_results()
remove_adjustment(10)
print('------------------')
adjust_percent_up(20)
print_results()
remove_adjustment(20)



Equation: straight_line
Simple % error: 9.090947636389341%
Simpsons % error: 9.090909090909179%
Simple Error / Simpsons Error : 1.000004240002818
Equation: quadratics
Simple % error: 9.090966909118956%
Simpsons % error: 9.090909090913922%
Simple Error / Simpsons Error : 1.0000063600025537
Equation: exponentials
Simple % error: 9.09110181817426%
Simpsons % error: 9.090909090907985%
Simple Error / Simpsons Error : 1.0000211999992903
Equation: sine
Simple % error: 9.090852079784636%
Simpsons % error: 9.09090909090941%
Simple Error / Simpsons Error : 0.9999937287762749
------------------
Equation: straight_line
Simple % error: 16.666702000023562%
Simpsons % error: 16.666666666666746%
Simple Error / Simpsons Error : 1.000002120001409
Equation: quadratics
Simple % error: 16.666719666692362%
Simpsons % error: 16.66666666667108%
Simple Error / Simpsons Error : 1.000003180001277
Equation: exponentials
Simple % error: 16.666843333326398%
Simpsons % error: 16.666666666665648%
Simple Error / Simps

So is it the case that Simpsons really is better than the simple integration technique given the roughness of our estimated values? 